In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

## Investigate data

In [2]:
df = pd.read_csv("covid_data.csv")

In [3]:
df.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306429 entries, 0 to 306428
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SNo              306429 non-null  int64  
 1   ObservationDate  306429 non-null  object 
 2   Province/State   228329 non-null  object 
 3   Country/Region   306429 non-null  object 
 4   Last Update      306429 non-null  object 
 5   Confirmed        306429 non-null  float64
 6   Deaths           306429 non-null  float64
 7   Recovered        306429 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 18.7+ MB


## Data cleaning

In [5]:
# Rename columns for convenience
df.columns = ['serial', 'obsv_date', 'province_state', 'country_region', 'last_update', 'confirmed', 'deaths', 'recovered']

# Set serial number as index
df = df.set_index('serial')

# Convert floats to integers
df.iloc[:,-3:] = df.iloc[:,-3:].astype(int)

# Strip whitespace
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Rename values
df = df.replace({'country_region': {"Bahamas, The": "Bahamas", 
                                    "The Bahamas": "Bahamas",
                                    "(\'St. Martin\',)": "St. Martin",
                                    "UK": "United Kingdom",
                                    "US": "United States", 
                                    " Azerbaijan": "Azerbaijan",
                                    "Gambia, The": "Gambia",
                                    "occupied Palestinian territory": "Occupied Palestinian Territory"},
                 'province_state': {"Grand Princess Cruise Ship": "Grand Princess"}})

# Remove duplicates
df = df.drop_duplicates()

# Remove non country/regions
df = df[df['country_region'] != 'MS Zaandam']

# Convert observation_date and last_update to datetime format
df['obsv_date'] = pd.to_datetime(df['obsv_date'])
df['last_update'] = pd.to_datetime(df['last_update'])

# Remove rows with all zeros
df = df[~((df['confirmed'] == 0) & (df['deaths'] == 0) & (df['recovered'] == 0))]

# Check for outliers
df[(df['confirmed'] < 0) | (df['deaths'] < 0) | (df['recovered'] < 0)]

# Drop Colombia:Unknown data
df = df[~((df['country_region'] == 'Colombia') & (df['province_state'] == "Unknown"))]

### Extract and clean data for India

In [ ]:
df = df[df['country_region'] == 'India']
df = df.sort_values(['province_state', 'obsv_date'])
df['lag'] = df['confirmed'].shift(1).fillna(0)
df['New_Cases'] = df['confirmed'] - df['lag']
df = df[['province_state', 'obsv_date', 'confirmed', 'New_Cases']]

index = df[df['obsv_date'] == '2020-06-10'].index
for x in index:
    df.loc[x, 'New_Cases'] = 0

df.iloc[:,-2:] = df.iloc[:,-2:].astype(int)

In [ ]:
df.to_csv('india_data.csv')

After cleaning the given data, I found a daily-updated json timeseries provided by Github user 'pomber' (https://github.com/pomber/covid19) who transforms the JHU data into json format. After checking that their data matches my cleaned data, I decided to use the json to obtain an updated version of the data. However, I am keeping the extracted India data as the json does not contain Indian state info. This state info will be used for the India map in the dashboard. 

Since the Github data is cumulative, I also made a version of the data that is non-cumulative (i.e. giving the number of new cases/deaths/recoveries every day). See 'extract_json.ipynb' for the data retrieval. 

## Cleaning updated JHU global data (cumulative data)

In [8]:
covid_json = pd.read_csv("covid_data_from_json.csv")
covid_json['Date'] = pd.to_datetime(covid_json['Date'])

covid_json = covid_json.replace({'Country': {
                                    "UK": "United Kingdom",
                                    "US": "United States", 
                                    "Taiwan*": "Taiwan"}})

covid_json = covid_json.groupby(['Country', 'Date']).sum(['Confirmed', 'Deaths', 'Recovered'])
covid_json = covid_json.sort_index(level = ['Country', 'Date'])

# Remove non-countries from countries
covid_json = covid_json.drop(index = "Summer Olympics 2020")
covid_json = covid_json.drop(index = "Diamond Princess")
covid_json = covid_json.drop(index = "MS Zaandam")

# Change 'Recovered' data from Aug5-present to be NaN rather than 0
for x in covid_json.index.get_level_values(0).unique():
    covid_json.loc[(x, "2021-08-05"):(x, "2021-10-13"), 'Recovered'] = np.NaN

# Change 'Recovered' data from Dec14-present to be NaN rather than 0 for United States
covid_json.loc[("United States", "2020-12-14"):("United States", "2021-10-13"), 'Recovered'] = np.NaN

In [9]:
covid_json

Confirmed  Deaths  Recovered
Country     Date                                    
Afghanistan 2020-01-22          0       0        0.0
            2020-01-23          0       0        0.0
            2020-01-24          0       0        0.0
            2020-01-25          0       0        0.0
            2020-01-26          0       0        0.0
...                           ...     ...        ...
Zimbabwe    2021-10-09     131762    4636        NaN
            2021-10-10     131796    4637        NaN
            2021-10-11     131875    4643        NaN
            2021-10-12     132016    4645        NaN
            2021-10-13     132108    4648        NaN

[122414 rows x 3 columns]

In [10]:
covid_json_recov = covid_json[~covid_json['Recovered'].isna()]

In [11]:
covid_json.to_csv("covid_final_cumulative.csv")
covid_json_recov.loc[:, 'Recovered'].to_csv("covid_final_cumulative_recov.csv")

## Cleaning updated JHU data (non-cumulative data)

In [24]:
daily_cases = pd.read_csv("covid_new_cases_from_json.csv")
daily_cases['Date'] = pd.to_datetime(daily_cases['Date'])

daily_cases = daily_cases.replace({'Country': {
                                    "UK": "United Kingdom",
                                    "US": "United States", 
                                    "Taiwan*": "Taiwan"}})

daily_cases = daily_cases.groupby(['Country', 'Date']).sum(['Confirmed', 'Deaths', 'Recovered'])
daily_cases = daily_cases.sort_index(level = ['Country', 'Date'])

# Remove non-countries from countries
daily_cases = daily_cases.drop(index = "Summer Olympics 2020")
daily_cases = daily_cases.drop(index = "Diamond Princess")
daily_cases = daily_cases.drop(index = "MS Zaandam")

daily_cases.loc['Turkey'].loc['2020-12-10', 'New_Confirmed'] = 30424

In [25]:
daily_cases

New_Confirmed  New_Deaths  New_Recovered
Country     Date                                                
Afghanistan 2020-01-22              0           0              0
            2020-01-23              0           0              0
            2020-01-24              0           0              0
            2020-01-25              0           0              0
            2020-01-26              0           0              0
...                               ...         ...            ...
Zimbabwe    2021-10-09             57           2              0
            2021-10-10             34           1              0
            2021-10-11             79           6              0
            2021-10-12            141           2              0
            2021-10-13             92           3              0

[121152 rows x 3 columns]

In [26]:
daily_cases.to_csv("covid_final_new_cases.csv")